In [13]:
import numpy as np
import cv2
import time
from IPython.display import clear_output
import os
import requests

WIND_X = 640
WIND_Y = 480

BACKGROUND_COLOR = (150, 160, 170)

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

get_available_cameras ()

[0, 1]

In [14]:
def calibration(x_array, y_array):
        rectangle_calibrovka = [0, 0, 0, 0]
        ###find top position###
        top = [0, 0]
        for i in range(len(y_array)):
            if y_array[i] > top[1]:
                top[0] = x_array[i]
                top[1] = y_array[i]
        ###find left position###
        left = [0, 0]
        left[0] = 10e10
        for i in range(len(x_array)):
            if x_array[i] < left[0]:
                left[0] = x_array[i]
                left[1] = y_array[i]
        ###find right position###
        right = [0, 0]
        right[0] = 0
        for i in range(len(x_array)):
            if x_array[i] > right[0]:
                right[0] = x_array[i]
                right[1] = y_array[i]
        ###find botom position###
        bottom = [0, 0]
        bottom[1] = 10e10
        for i in range(len(y_array)):
            if y_array[i] < bottom[1]:
                bottom[0] = x_array[i]
                bottom[1] = y_array[i]
        #cv2.rectangle(img,(left[0], up[1]),(right[0]+dev1, down[1]+dev2),(255,0,255),3)
            
        rectangle_calibrovka[0] = left[0]
        rectangle_calibrovka[1] = top[1]
        rectangle_calibrovka[2] = right[0]#+size_cell_x
        rectangle_calibrovka[3] = bottom[1]#+size_cell_y
                    
        return rectangle_calibrovka

def detect_marker (img, color):
    #color = [hsv_min, hsv_max]
    
    # преобразуем RGB картинку в HSV модель
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # применяем цветовой фильтр

    thresh = cv2.inRange(hsv, color[0], color[1])
    
    # вычисляем моменты изображения
    moments = cv2.moments(thresh, 1)
    dM01 = moments['m01']
    dM10 = moments['m10']
    dArea = moments['m00']
    # будем реагировать только на те моменты,
    # которые содержать больше 100 пикселей
    
    marker_coord = [0, 0]
    if dArea > 100:
        x = int(dM10 / dArea)
        y = int(dM01 / dArea)
        marker_coord[0] = x
        marker_coord[1] = y
        #cv2.circle(img, (x, y), 10, (0,0,255), -1)
    return marker_coord

def find_marker_place (img, marker_coord, size_cell_x, size_cell_y, color):
    coord_rectang = [0, 0, 0, 0]
    k_i = 0
    l_j = 0
    while k_i <= img.shape[1]:
        while l_j <= img.shape[0]:
            if x < k_i + size_cell_x and x > k_i and y < l_j + size_cell_y and y > l_j:
                coord_rectang[0] = k_i
                coord_rectang[1] = l_j
                coord_rectang[2] = k_i+size_cell_x
                coord_rectang[3] = l_j+size_cell_y
                #cv2.rectangle(img,(k_i, l_j),(k_i+dev1, l_j+dev2),(0,255,255),3)                  
            l_j+= dev2
        k_i+= dev1
        l_j = 0
        
    return coord_rectang

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    
    lst = list(dim)
    lst[0] = int(lst[0])
    dim = tuple(lst)
    
    resized = cv2.resize(image, dim, interpolation = inter)
    
    # return the resized image
    return resized


def find_movement_of_man (curr_frame, back_frame, connectivity = 4):
    
    #shape
    shape = curr_frame.shape
    
    #make a photo of curr_frame
    new_frame = cv2.imwrite('opencv1.png', curr_frame)
    
    #make a resize
    back_frame_resize = image_resize(back_frame , width = shape[1]/4, height = shape[0]/4)
    new_frame_resize = image_resize(new_frame , width = shape[1]/4, height = shape[0]/4)
    
    #filter
    filtered_back_frame = cv2.blur (back_frame, (9, 9))
    filtered_new_frame = cv2.blur (new_frame, (9, 9))
    
    #difference between photos
    diff_between = cv2.absdiff(filtered_back_frame,filtered_new_frame)
    

    img_grey = cv2.cvtColor(diff_between, cv2.COLOR_BGR2GRAY)
    
    gaussian_blur = cv2.GaussianBlur(img_grey, (5,5), 0)
    
    #binarization
    _, frame = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    kernel = np.ones((10,10),np.uint8)
    erosia = cv2.erode(frame,kernel,iterations = 1)
    
    frame_end = image_resize(erosia , width = sh[1], height = sh[0])
    
    #find the lagest component of connectivity
    output = cv2.connectedComponentsWithStats(d, connectivity, cv2.CV_32S)
    
    #rectangle around area
    
    new_rectangle_coord = [0, 0, 0, 0]
    for i in range(output[0]):
            if output[2][i][4] >= 5000: 
                new_rectangle_coord[0] = output[2][i][0]
                new_rectangle_coord[1] = output[2][i][1]
                new_rectangle_coord[2] = output[2][i][0] + output[2][i][2]
                new_rectangle_coord[3] = output[2][i][1] + output[2][i][3]
                #cv2.rectangle(d, (output[2][i][0], output[2][i][1]), 
                ####(output[2][i][0] + output[2][i][2], output[2][i][1] + output[2][i][3]), (255,0,255), 2)
    
    return new_rectangle_coord

In [37]:
cv2.namedWindow ("controller", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("controller", (WIND_X * 2, WIND_Y))

cam = cv2.VideoCapture (1)
ret, initial_frame = cam.read ()

background = initial_frame.copy ()

#measurements_num = 6 #measurements per series
#series_num = 2

time_limit     = 10000
measuring_mode = False
movement_type  = 1

start_time         = 0
current_time       = 0
time_passed        = 0
robot_command_sent = False

relax_mode    = False
relax_timeout = 5

marker_color_min = (0, 140, 60)
marker_color_max = (30, 160, 90)

calibration_mode = False

curr_measurement_finished = False

curr_time = time.time ()
files_common_part = str (curr_time)

fourcc1 = cv2.VideoWriter_fourcc (*'XVID')
stream  = cv2.VideoWriter ("log/" + files_common_part + 'stream' + '.avi', fourcc1, 20.0, (WIND_X, WIND_Y))

fourcc2 = cv2.VideoWriter_fourcc (*'XVID')
cv      = cv2.VideoWriter ("log/" + files_common_part + 'cv' + '.avi', fourcc2, 20.0, (WIND_X, WIND_Y))

log = open ("log/" + files_common_part + "log.txt", 'w')

calib_marker_x = []
calib_marker_y = []

ip = "http://192.168.43.201:"
port = "9559"

rect_calib = [100, 100, 300, 250]

while (True):
    ret, frame = cam.read ()
    
    stream.write (frame)
    cv.write     (frame)
    log.write ("ffuu")
    
    cv2.waitKey (1)
    
    frame_resized = cv2.resize (frame, (WIND_X, WIND_Y))
    
    current_time = time.time ()
    time_passed = current_time - start_time
    
    marker = detect_marker (frame, (marker_color_min, marker_color_max))
    
    if (measuring_mode == True and relax_mode == False):
        #send command to robot
        if (robot_command_sent == False):
            #wget ....
            #os.system ("wget " + '"' + ip + port + "/?action=/hands_up&text=kek")

            r = requests.get ("http://192.168.1.250:9559/?action=red&text=qwer")
            
            #play sound
            robot_command_sent = True
        
        marker = detect_marker (frame, (marker_color_min, marker_color_max))
        
        #if (cv says "done"):
        if (marker [1] < rect_calib [1] + (rect_calib [3] - rect_calib [1]) / 3):
            print ("success")
            #send command to haptics
            curr_measurement_finished = True
        
        if (time_passed > time_limit):
            curr_measurement_finished = True
    
    if (curr_measurement_finished == True):
        #hands down to robot
        curr_measurement_finished = False
        relax_mode = True
        start_time = current_time
        time_passed = 0
    
    if (relax_mode == True):
        print ("relax")
        
        if (time_passed > relax_timeout):
            relax_mode = False
            start_time = current_time
            time_passed = 0
        
    analysis = frame_resized.copy ()
    cv2.rectangle (analysis, (0, 0), (WIND_X, WIND_Y), BACKGROUND_COLOR, thickness = -1)
    
    cv2.rectangle (frame_resized, (int (rect_calib [0]), int (rect_calib [1])),
                   (int (rect_calib [2]), int (rect_calib [3])),
                   (100, 200, 100), thickness = 5)
    
    cv2.circle (frame_resized, (marker [0], marker [1]), 10, (0,0,255), -1)

    if (calibration_mode == True):
        marker = detect_marker (frame, (marker_color_min, marker_color_max))
        calib_marker_x.append (marker [0])
        calib_marker_y.append (marker [1])
        
        cv2.circle (analysis, (marker [0], marker [1]), 10, (0,0,255), -1)

    #text
    #time left to the end of the measurement
    #number of current measurement/total number of measurements
    #type of the movement
    
    #записать картинки в файлы
    
    output = np.concatenate ((frame_resized, analysis), axis = 1)    
    #output = np.concatenate ((frame_resized, background), axis = 1)    

    cv2.imshow ("controller", output)
    
    time.sleep (0.01)
    keyb = cv2.waitKey (1)
    
    if (keyb & 0xFF == ord ('q')):
        break

    elif (keyb & 0xFF == ord ('s')):
        measuring_mode = True
        start_time = time.time ()

    elif (keyb & 0xFF == ord ('f')):
        measuring_mode = False

    elif (keyb & 0xFF == ord ('1')):
        movement_type = 1

    elif (keyb & 0xFF == ord ('2')):
        movement_type = 2
    
    elif (keyb & 0xFF == ord ('3')):
        movement_type = 3
    
    #take background photo
    elif (keyb & 0xFF == ord ('b')):
        background = frame
    
    #start calibration
    elif (keyb & 0xFF == ord ('c')):
        calibration_mode = True
    
    #stop calibration
    elif (keyb & 0xFF == ord ('v')):
        calibration_mode = False
        rect_calib = calibration (calib_marker_x, calib_marker_y)
        calib_marker_x.clear ()
        calib_marker_y.clear ()
    
cam.release ()

#закрыть файлы
stream.release ()
cv.release     ()
log.close    ()

cv2.destroyAllWindows()